In [289]:
import matplotlib.pyplot as plt
%matplotlib widget

from scipy.optimize import curve_fit
from scipy.optimize import minimize
import numpy as np

import exoplanet as xo
import pymc3 as pm
import pymc3_ext as pmx
import aesara_theano_fallback.tensor as tt
from celerite2.theano import terms, GaussianProcess

In [486]:
def func(x, a, b, c, d, e, f, g, h):
    return a + b*x + c*x**2.0 + d*x**3.0 + e*x**4.0 + f*x**5.0 + g*x**6.0 + h*x**6.0

In [291]:
fake_y, fake_x = np.array([0.5, 0.3, -0.2, -2.25, 1.2, 4.5]), np.array([1, 1.5, 2.25, 6.0, 10, 15.5])
fake_y_err = np.ones_like(fake_y) * 0.1

sampled_x = np.linspace(min(fake_x), max(fake_x), 1000)

popt, pcov = curve_fit(func, fake_x, fake_y, p0=[0.3, 0.6, -0.5, 0.09, -0.05], sigma=fake_y_err)

perr = np.sqrt(np.diag(pcov))

popt_m1 = popt - perr
popt_p1 = popt + perr

In [292]:
import celerite2
from celerite2 import terms

x1 = np.linspace(1, 2.25, 100)
x2 = np.array([6.0, 10, 15.5])
x = np.concatenate((x1, x2))
yerr = np.random.uniform(0.08, 0.22, len(x))
y = func(x, *popt)

kernel = terms.Matern32Term(sigma=3.0, rho=5.0)

# Setup the GP
gp = celerite2.GaussianProcess(kernel, mean=np.average(y))
gp.compute(x)#, yerr=yerr)

mu, variance = gp.predict(y, t=sampled_x, return_var=True)
sigma = np.sqrt(variance)

In [293]:
fig, ax = plt.subplots(figsize=(10, 4))

ax.set_ylim(-13.1, 15.3)
ax.set_xlabel("some time $t$")
ax.set_ylabel("some data $y$")


ax.scatter(fake_x, fake_y, s=50, color='blue', zorder=10)
#plt.savefig("../Talks/Invited Yale Talk/GP Explaination/data.png", bbox_inches='tight', dpi=400)

ax.plot(sampled_x, func(sampled_x, *popt), 'k--')
#plt.savefig("../Talks/Invited Yale Talk/GP Explaination/fit.png", bbox_inches='tight', dpi=400)

#ax.fill_between(sampled_x, y1=func(sampled_x, *popt_m1), y2=func(sampled_x, *popt_p1), color="C0", alpha=0.2)
#plt.savefig("../Talks/Invited Yale Talk/GP Explaination/fit_w_errs.png", bbox_inches='tight', dpi=400)

ax.fill_between(sampled_x, mu - 3*sigma, mu + 3*sigma, color="C0", alpha=0.2)
#plt.savefig("../Talks/Invited Yale Talk/GP Explaination/fit_w_gp.png", bbox_inches='tight', dpi=400)

plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [294]:
plt.close()

In [648]:
np.random.seed(42)

t = np.sort(
    np.append(
        np.random.uniform(0, 3.8, 57),
        np.random.uniform(5.5, 10, 68),
    )
)  # The input coordinates must be sorted
yerr = np.random.uniform(0.08, 0.22, len(t))
y = (
    0.2 * (t - 5)
    + np.sin(3 * t + 0.1 * (t - 5) ** 2)
    + yerr * np.random.randn(len(t))
)

true_t = np.linspace(0, 10, 500)
true_y = 0.2 * (true_t - 5) + np.sin(3 * true_t + 0.1 * (true_t - 5) ** 2)

def fake_func(t, c, d):
    return 0.15 * (t - 5.0) + np.sin(c * t + d * (t - 5.0) ** 2)

def fake_func2(t, a, b, c, d, e, f, g, h, j, k):
    return a*np.sin(b*t - c) + d*np.cos(e*t - f) + g*np.sin(h*t - c)**2 + j*np.cos(k*t - f)**2

def fake_func3(t, a, b, c, d, e, f, g, h, i, j):
    return a*np.sin(b*t - c)**2 + d*np.cos(e*t - f)**2 + g*(t-h) + i*(t-j)**2

In [653]:
popt, pcov = curve_fit(fake_func, t, y, p0=[3.0, 0.2], sigma=yerr, absolute_sigma=True)

popt2, pcov2 = curve_fit(fake_func2, t, y, sigma=yerr, absolute_sigma=True, maxfev=10000)
popt3, pcov3 = curve_fit(fake_func3, t, y, sigma=yerr, absolute_sigma=True)

In [654]:
# Quasi-periodic term
term1 = terms.SHOTerm(sigma=1.0, rho=1.0, tau=10.0)

# Non-periodic component
term2 = terms.SHOTerm(sigma=1.0, rho=5.0, Q=0.25)
kernel = term1 + term2

# Setup the GP
gp = celerite2.GaussianProcess(kernel, mean=0.0)
gp.compute(t, yerr=yerr)

initial_mu, initial_variance = gp.predict(y, t=true_t, return_var=True)
initial_sigma = np.sqrt(initial_variance)

def set_params(params, gp):
    gp.mean = params[0]
    theta = np.exp(params[1:])
    gp.kernel = terms.SHOTerm(
        sigma=theta[0], rho=theta[1], tau=theta[2]
    ) + terms.SHOTerm(sigma=theta[3], rho=theta[4], Q=0.25)
    gp.compute(t, diag=yerr ** 2 + theta[5], quiet=True)
    return gp


def neg_log_like(params, gp):
    gp = set_params(params, gp)
    return -gp.log_likelihood(y)


initial_params = [0.0, 0.0, 0.0, np.log(10.0), 0.0, np.log(5.0), np.log(0.01)]
soln = minimize(neg_log_like, initial_params, method="L-BFGS-B", args=(gp,))
opt_gp = set_params(soln.x, gp)

mu, variance = opt_gp.predict(y, t=true_t, return_var=True)
sigma = np.sqrt(variance)

In [665]:
fig, ax = plt.subplots(figsize=(10, 4))

#ax.plot(true_t, true_y, "k", lw=1.5, alpha=0.3, label='Truth')

ax.errorbar(t, y, yerr=yerr, fmt=".k", capsize=0, label='Data')

#ax.plot(true_t, fake_func(true_t, *popt), 'k--', label='Traditional Regression')

ax.plot(true_t, fake_func(true_t, *popt), 'k--', lw=1.0, alpha=0.35, label='Traditional Regression')
#ax.plot(true_t, fake_func2(true_t, *popt2), 'k--', lw=1.0, alpha=0.35)
#ax.plot(true_t, fake_func3(true_t, *popt3), 'k--', lw=1.0, alpha=0.35)
#for n in [12, 13, 14, 15, 16, 17, 18, 19, 20]:
#    z = np.polyfit(t, y, n)
#    p = np.poly1d(z)
#
#    ax.plot(true_t, p(true_t), 'k--', lw=1.0, alpha=0.35)#, label='Traditional Regression')

#ax.plot(true_t, initial_mu, label="Initial GP")
#ax.fill_between(true_t, initial_mu - initial_sigma, initial_mu + initial_sigma, color="C0", alpha=0.2)

#ax.plot(true_t, mu, label="Final GP")
#ax.fill_between(true_t, mu - sigma, mu + sigma, color="C0", alpha=0.2)

ax.set_xlabel("$t$ (day)")
ax.set_ylabel("Data (ppm)")

ax.legend(loc='upper left', fontsize=16, markerscale=1.0)

ax.set_xlim(0, 10)
ax.set_ylim(-2.5, 2.5)

#plt.savefig("../Talks/Invited Yale Talk/GP Explaination/gp_ex_truth.png", bbox_inches='tight', dpi=400)
#plt.savefig("../Talks/Invited Yale Talk/GP Explaination/gp_ex_data.png", bbox_inches='tight', dpi=400)
#plt.savefig("../Talks/Invited Yale Talk/GP Explaination/gp_ex_data_only.png", bbox_inches='tight', dpi=400)
#plt.savefig("../Talks/Invited Yale Talk/GP Explaination/gp_ex_traditional.png", bbox_inches='tight', dpi=400)
plt.savefig("../Talks/Invited Yale Talk/GP Explaination/gp_ex_multi_traditional1.png", bbox_inches='tight', dpi=400)
#plt.savefig("../Talks/Invited Yale Talk/GP Explaination/gp_ex_multi_traditional2.png", bbox_inches='tight', dpi=400)
#plt.savefig("../Talks/Invited Yale Talk/GP Explaination/gp_ex_multi_traditional3.png", bbox_inches='tight', dpi=400)
#plt.savefig("../Talks/Invited Yale Talk/GP Explaination/gp_ex_multi_traditional.png", bbox_inches='tight', dpi=400)
#plt.savefig("../Talks/Invited Yale Talk/GP Explaination/gp_initial_gp.png", bbox_inches='tight', dpi=400)
#plt.savefig("../Talks/Invited Yale Talk/GP Explaination/gp_final_gp.png", bbox_inches='tight', dpi=400)

#plt.tight_layout()
#plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [661]:
plt.close()